In [ ]:
import mesh2sdf
import trimesh

filename = '../model/bunny.obj'
mesh = trimesh.load(filename, force='mesh')
mesh.show()

In [27]:
mesh.is_watertight

False

In [28]:
mesh.vertices[:,0]+=0.3
mesh.vertices[:,0]

TrackedArray([0.277368, 0.224826, 0.222761, ..., 0.272277, 0.227981,
              0.255964])

In [29]:
size = 128#sdf's shape is (size,size,size)
level = 2 / size

In [ ]:
mesh_scale = 0.8
vertices=mesh.vertices
bbmax=vertices.max(0)
bbmin=vertices.min(0)
center=(bbmin+bbmax)*0.5
print((bbmax - bbmin).max())

scale = 2.0 * mesh_scale / (bbmax - bbmin).max()

mesh.vertices = (vertices - center) * scale
mesh.show()

In [33]:
# fix mesh
import time
vertices=mesh.vertices*0.5
t0 = time.time()
sdf, mesh = mesh2sdf.compute(vertices, mesh.faces, size, fix=True, level=level, return_mesh=True)
t1 = time.time()

In [ ]:
# output
import numpy as np
mesh.vertices = mesh.vertices / scale + center
mesh.show()

### Visualize SDF

In [36]:
import os
import sys
import matplotlib.pyplot as plt
import numpy as np
import skimage.measure

In [37]:
# sdf = np.load('../model/bunny.npy')
print(sdf.shape)
size = sdf.shape[0]
print(sdf.max(), sdf.min())

(128, 128, 128)
1.4496022 -0.03047628


In [38]:
# #Not necessary, used for debug.
# mesh_scale = 0.8
# levels = [-0.02, 0.0, 0.02]
# # extract level sets
# for i, level in enumerate(levels):
vtx, faces, _, _ = skimage.measure.marching_cubes(sdf, 0.0)

#   vtx = vtx * (mesh_scale * 2.0 / size) - 1.0
#   mesh = trimesh.Trimesh(vtx, faces)
#   mesh.export(os.path.join('./', 'l%.2f.obj' % level))

In [40]:
vtx.shape

(17559, 3)

In [ ]:
mesh=trimesh.Trimesh(vtx,faces)
mesh.show()

In [ ]:
# draw image
for i in range(size):
  array_2d = sdf[:, :, i]

  num_levels = 6
  fig, ax = plt.subplots(figsize=(2.75, 2.75), dpi=300)
  levels_pos = np.logspace(-2, 0, num=num_levels)  # logspace
  levels_neg = -1. * levels_pos[::-1]
  levels = np.concatenate((levels_neg, np.zeros((0)), levels_pos), axis=0)
  colors = plt.get_cmap("Spectral")(np.linspace(0., 1., num=num_levels*2+1))

  sample = array_2d
  # sample = np.flipud(array_2d)
  CS = ax.contourf(sample, levels=levels, colors=colors)

  ax.contour(sample, levels=levels, colors='k', linewidths=0.1)
  ax.contour(sample, levels=[0], colors='k', linewidths=0.3)
  ax.axis('off')

  plt.savefig(os.path.join('./frames', '%03d.png' % i))

In [ ]:
# draw image
for i in range(64):
  array_2d = sdf[-64:, -64:, -i]

  num_levels = 6
  fig, ax = plt.subplots(figsize=(2.75, 2.75), dpi=300)
  levels_pos = np.logspace(-2, 0, num=num_levels)  # logspace
  levels_neg = -1. * levels_pos[::-1]
  levels = np.concatenate((levels_neg, np.zeros((0)), levels_pos), axis=0)
  colors = plt.get_cmap("Spectral")(np.linspace(0., 1., num=num_levels*2+1))

  sample = array_2d
  # sample = np.flipud(array_2d)
  CS = ax.contourf(sample, levels=levels, colors=colors)

  ax.contour(sample, levels=levels, colors='k', linewidths=0.1)
  ax.contour(sample, levels=[0], colors='k', linewidths=0.3)
  ax.axis('off')

  plt.savefig(os.path.join('./frames', '%03d.png' % i))

In [21]:
x=sdf[-64:,-64:,-64:]
type(x)

numpy.ndarray